# First steps with Claude throught Bedrock

## The most simple approach

In [20]:
import os
import boto3
from dotenv import load_dotenv

In [21]:
load_dotenv() # load the environment variables

True

In [22]:
profile_name=os.getenv("PROFILE_NAME") #  I currently log in using SSO.

In [23]:
session = boto3.session.Session(profile_name=profile_name, region_name="us-west-2")

In [6]:
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

user_message = {
    "role": "user",
    "content": [
        {"text": "What's 1+1?"}
    ]
}

response = client.converse(
    modelId = model_id,
    messages = [user_message]
)

In [7]:
response["output"]["message"] 

{'role': 'assistant', 'content': [{'text': '1 + 1 = 2'}]}

## Multi-turn approach

In [8]:
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def add_user_message(messages: list, text: str):
        
    user_message = {
        "role": "user",
        "content": [
            {"text": text}
        ]
    }

    messages.append(user_message)
    

def add_assistant_message(messages: list, text: str):
        
    assistant_message = {
        "role": "assistant",
        "content": [
            {"text": text}
        ]
    }

    messages.append(assistant_message) 

def chat(messages: list):
    response = client.converse(
        modelId = model_id,
        messages = messages
    )

    return response["output"]["message"]["content"][0]["text"]

In [9]:
# started with an empty list of messages where we maintaining the context
messages = []

# initial user question
add_user_message(messages, "What's 1+1?")

# pass the list of messages into chat
answer = chat(messages)

# take the answer from the assistant and added it into the message list  
add_assistant_message(messages, answer)

# followup question
add_user_message(messages, "added 3 more, what's the result?")

# chat again
answer = chat(messages)

answer

'If we add 3 more to the previous result of 2:\n\n2 + 3 = 5'

### Query Processing

In [10]:
messages=[]
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def process_query(query: str):
    
    add_user_message(messages, query)
    answer = chat(messages)
    add_assistant_message(messages, answer)
    return answer
                

# Chat Loop

In [18]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [19]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  What's 1+1?



Assistant: 1 + 1 = 2


Query:  add 3 



Assistant: 2 + 3 = 5


Query:  thank you. 



Assistant: You're welcome! Happy to help with the math.


Query:  quit


# System Prompt

## First Approach

In [55]:
from typing import Optional

def add_user_message(messages: list, text: str):
        
    user_message = {
        "role": "user",
        "content": [
            {"text": text}
        ]
    }

    messages.append(user_message)
    

def add_assistant_message(messages: list, text: str):
        
    assistant_message = {
        "role": "assistant",
        "content": [
            {"text": text}
        ]
    }

    messages.append(assistant_message) 

def chat(messages: list, system_prompt:Optional[str] = None):
    params = {"modelId": model_id, "messages": messages}

    if system_prompt:
        params["system"] = [{"text": system_prompt}]
            
    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

In [56]:
messages=[]
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def process_query(query: str, system_prompt:Optional[str] = None):
    
    add_user_message(messages, query)
    answer = chat(messages, system_prompt)
    add_assistant_message(messages, answer)
    return answer
                

## Without System prompt

In [63]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [64]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  How do I host a Postgres database?



Assistant: There are several ways to host a PostgreSQL database. Here are the main options:

## 1. **Cloud-Managed Services** (Recommended for most use cases)

### AWS RDS PostgreSQL
```bash
# Using AWS CLI
aws rds create-db-instance \
    --db-instance-identifier mypostgres \
    --db-instance-class db.t3.micro \
    --engine postgres \
    --master-username myuser \
    --master-user-password mypassword \
    --allocated-storage 20
```

### Google Cloud SQL
```bash
gcloud sql instances create my-postgres-instance \
    --database-version=POSTGRES_15 \
    --tier=db-f1-micro \
    --region=us-central1
```

### Azure Database for PostgreSQL
```bash
az postgres server create \
    --resource-group myResourceGroup \
    --name mydemoserver \
    --location westus \
    --admin-user myadmin \
    --admin-password mypassword \
    --sku-name GP_Gen5_2
```

## 2. **Self-Hosted on Virtual Private Server (VPS)**

### On Ubuntu/Debian:
```bash
# Update system
sudo apt update && sudo apt upgra

Query:  quit


## System prompt GCP expert

In [57]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query, system_prompt="You are a GCP expert")
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [58]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  How do I host a Postgres database?



Assistant: There are several ways to host a PostgreSQL database on Google Cloud Platform. Here are the main options:

## 1. **Cloud SQL for PostgreSQL** (Recommended for most use cases)
This is Google's fully managed PostgreSQL service.

### Setup Steps:
```bash
# Create a Cloud SQL instance
gcloud sql instances create my-postgres-instance \
    --database-version=POSTGRES_15 \
    --tier=db-f1-micro \
    --region=us-central1 \
    --storage-type=SSD \
    --storage-size=10GB

# Create a database
gcloud sql databases create my-database \
    --instance=my-postgres-instance

# Create a user
gcloud sql users create myuser \
    --instance=my-postgres-instance \
    --password=mypassword
```

### Benefits:
- Automated backups and updates
- High availability options
- Automatic scaling
- Built-in security features
- Point-in-time recovery

## 2. **AlloyDB for PostgreSQL** (For enterprise workloads)
Google's latest PostgreSQL-compatible database service with enhanced performance.

```bash

Query:  quit


## System prompt AWS expert by default

In [61]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query, system_prompt="You are a AWS expert")
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [62]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  How do I host a Postgres database?



Assistant: There are several ways to host a PostgreSQL database on AWS. Here are the main options:

## 1. **Amazon RDS for PostgreSQL** (Recommended for most use cases)
This is AWS's fully managed PostgreSQL service.

### Setup Steps:
```bash
# Using AWS CLI
aws rds create-db-instance \
    --db-instance-identifier mypostgres \
    --db-instance-class db.t3.micro \
    --engine postgres \
    --master-username myuser \
    --master-user-password mypassword \
    --allocated-storage 20 \
    --vpc-security-group-ids sg-12345678 \
    --db-subnet-group-name default
```

### Benefits:
- Automated backups and patching
- Multi-AZ deployments for high availability
- Read replicas for scaling
- Point-in-time recovery
- Built-in monitoring with CloudWatch

## 2. **Amazon Aurora PostgreSQL** (For high-performance workloads)
AWS's cloud-native, PostgreSQL-compatible database.

```bash
# Create Aurora cluster
aws rds create-db-cluster \
    --db-cluster-identifier my-aurora-cluster \
    --engin

Query:  quit
